In [1]:
import json
import altair as alt
from altair import expr, datum
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import requests

In [108]:
LOCAL = False

if LOCAL:
    local_suffix = "_local"
else:
    local_suffix = ""

In [109]:
%%capture pwd
!pwd

In [110]:
# uid = "2021-05-05-which-firms-and-industries-have-been-most-affected-by-covid-update"  # article unique ID
uid = pwd.stdout.split("/")[-1].split("\r")[0]
eco_git_home = (
    "https://raw.githubusercontent.com/EconomicsObservatory/ECOvisualisations/main/"
)
eco_git_path = eco_git_home + "articles/" + uid + "/data/"
vega_embed = requests.get(eco_git_home + "guidelines/html/vega-embed.html").text
colors = json.loads(
    requests.get(eco_git_home + "guidelines/colors/eco-colors.json").content
)
category_color = json.loads(
    requests.get(eco_git_home + "guidelines/colors/eco-category-color.json").content
)
hue_color = json.loads(
    requests.get(eco_git_home + "guidelines/colors/eco-single-hue-color.json").content
)
mhue_color = json.loads(
    requests.get(eco_git_home + "guidelines/colors/eco-multi-hue-color.json").content
)
div_color = json.loads(
    requests.get(eco_git_home + "guidelines/colors/eco-diverging-color.json").content
)
config = json.loads(
    requests.get(eco_git_home + "guidelines/charts/eco-global-config.json").content
)
height = config["height"]
width = config["width"]
height, width

(300, 500)

# Fig 1

In [124]:
df = pd.read_csv("raw/ports_C3b.csv").dropna(axis=1)
df["Date"] = pd.to_datetime("01-" + df["Date"].astype(str), dayfirst=True)

In [125]:
f = "fig1_routes"
f1 = eco_git_path + f + ".csv"
df.to_csv("data/" + f + ".csv")
f += local_suffix
open("visualisation/" + f + ".html", "w").write(
    vega_embed.replace(
        "JSON_PATH", f1.replace("/data/", "/visualisation/").replace(".csv", ".json")
    )
)
if LOCAL:
    f1 = df
df.head()

,Route,Date,Value
0,Holyhead - Dublin,2020-09-01,5.8
1,Fishguard - Rosslare,2020-09-01,-4.0
2,Holyhead - Dublin,2020-10-01,3.1
3,Fishguard - Rosslare,2020-10-01,-2.8
4,Holyhead - Dublin,2020-11-01,12.1


In [126]:
f1

'https://raw.githubusercontent.com/EconomicsObservatory/ECOvisualisations/main/articles/2021-05-25-how-are-covid-19-and-brexit-affecting-ports-in-wales/data/fig1_routes.csv'

In [138]:
bars = (
    alt.Chart(f1)
    .encode(
        y=alt.Y(
            "Value:Q",
            stack=False,
            title="",
            axis=alt.Axis(
                grid=True,
                title="%",
                titleAnchor="start",
                labelColor=colors["eco-gray"],
                titleColor=colors["eco-gray"],
                tickColor=colors["eco-gray"],
                domainColor=colors["eco-gray"],
                titleFontSize=10,
                titleFontWeight="normal",
                ticks=False,
                labelAlign="left",
                labelBaseline="middle",
                labelPadding=-5,
                labelOffset=-10,
                titleX=25,
                titleY=273,
                titleBaseline="bottom",
                titleAngle=0,
                titleAlign="left",
                tickCount=5,
                format=".0f",
            ),
            scale=alt.Scale(domain=[-90, 45]),
        ),
        x=alt.X(
            "Date:T",
            title="",
            axis=alt.Axis(
                labelColor=colors["eco-gray"],
                titleColor=colors["eco-gray"],
                tickColor=colors["eco-gray"],
                domainColor=colors["eco-gray"],
                orient="top",
                # labelExpr="datum.Date=='2021-01'>-1?slice(datum.value,0,4)+' '+slice(datum.value,4):slice(datum.value,4)",
                labelAngle=0,
            ),
        ),
    )
    .transform_calculate(x1="datum.Value+5*(datum.Value<0?-1:1)")
)

bars1 = bars.mark_bar(
    size=15, xOffset=-8, color=colors["eco-turquiose"], opacity=0.8
).transform_filter("datum.Route=='Holyhead - Dublin'")
labels1 = bars1.mark_text(
    xOffset=-8,
    color=colors["eco-turquiose"],
).encode(text="Value:Q", y=alt.Y("x1:Q",scale=alt.Scale(domain=[-90, 45])))
labels2 = bars2.mark_text(
    xOffset=8,
    color=colors["eco-blue"],
).encode(text="Value:Q", y=alt.Y("x1:Q",scale=alt.Scale(domain=[-90, 45])))
bars2 = bars.mark_bar(
    size=15, xOffset=8, color=colors["eco-blue"], opacity=0.8
).transform_filter("datum.Route=='Fishguard - Rosslare'")
line = (
    alt.Chart(pd.DataFrame([{"x": "2020-08-10", "y": 0}, {"x": "2021-02-10", "y": 0}]))
    .mark_line(color=colors["eco-gray"], opacity=0.8)
    .encode(x="x:T", y=alt.Y("y:Q", scale=alt.Scale(domain=[-90, 45])))
)
labels = (
    alt.Chart(
        pd.DataFrame(
            [
                {
                    "x": "2020-12-21",
                    "y": -53,
                    "t": "Holyhead - Dublin",
                    "c": colors["eco-turquiose"],
                },
                {
                    "x": "2020-12-25",
                    "y": -72,
                    "t": "Fishguard - Rosslare",
                    "c": colors["eco-blue"],
                },
            ]
        )
    )
    .mark_text(xOffset=-5, align="right", size=10, baseline="middle")
    .encode(
        text="t",
        x="x:T",
        y=alt.Y("y:Q", scale=alt.Scale(domain=[-90, 45])),
        color=alt.Color("c:N", scale=None),
    )
)
layer = (
    (bars1 + bars2 + labels1 + labels2 + labels + line)
    .configure_view(stroke=None)
    .properties(title="")
    .properties(height=300, width=400)
)
layer.save("visualisation/" + f + ".json")
layer

alt.LayerChart(...)